In [4]:
import numpy as np
from datetime import datetime
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
import xarray as xr
import contextily as ctx
import shapely
import pint_xarray

In [5]:
#pip install pint_xarray

In [6]:
# we will look at january through april 
# predict discharge based on the snotel data
# spike in discharge means rain on snow, we will have 2 classes, so binary classification
# final goal: get a decision tree to predict discharge spikes based on snotel data 
# truth rain from snotel, infer it from discharge


# SWE: snow water equivalent: determines how much water the snowpack contains

In [ ]:
#pip install pint_xarray

In [7]:
# load in data 

snotel = xr.open_dataset('../../data/snotel/snotel_hourly_1980_2024.zarr')
usgs = xr.open_dataset('../../data/stream/usgs_instantaneous_1980_2024.zarr')

/opt/conda/lib/python3.11/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
/opt/conda/lib/python3.11/site-packages/zarr/codecs/vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)


In [8]:
snotel

<xarray.Dataset> Size: 16MB
Dimensions:                    (date: 218948, site: 2)
Coordinates:
  * date                       (date) datetime64[ns] 2MB 1987-10-22T06:00:00 ...
  * site                       (site) object 16B '817:WA:SNTL' '990:WA:SNTL'
Data variables:
    ACCUMULATED PRECIPITATION  (date, site) float64 4MB ...
    SWE                        (date, site) float64 4MB ...
    AIR TEMP                   (date, site) float64 4MB ...
    SNOWDEPTH                  (date, site) float64 4MB ...
Attributes:
    Description:  Snotel Data
    Date Range:   1980-2024
    Sites:        Thunder Basin, Beaver Creek
    Frequency:    Hourly

In [10]:
usgs

<xarray.Dataset> Size: 27MB
Dimensions:    (date: 1134647, site: 2)
Coordinates:
  * date       (date) datetime64[ns] 9MB 1989-10-01T07:15:00 ... 2024-01-02T0...
  * site       (site) object 16B '12172000' '12175500'
Data variables:
    DISCHARGE  (date, site) float64 18MB ...
Attributes:
    Description:  Streamflow Data
    Date Range:   1980-2024
    Sites:        Thunder Creek, Beaver Creek
    Frequency:    15-Min

In [11]:
usgs_df = usgs.to_dataframe()
usgs_df.head()

DISCHARGE
date                site               
1989-10-01 07:15:00 12172000        NaN
                    12175500      481.0
1989-10-01 07:30:00 12172000        NaN
                    12175500      481.0
1989-10-01 07:45:00 12172000        NaN

In [12]:
snotel_df = snotel.to_dataframe()

In [13]:
THUNDER_CREEK_USGS = ('12175500', 'Thunder Creek USGS Site')
BEAVER_CREEK_USGS = ('12172000', 'Beaver Creek USGS Site')

THUNDER_BASIN_SNOTEL = ('990:WA:SNTL', 'Thunder Basin Snotel Site')
BEAVER_CREEK_SNOTEL = ('817:WA:SNTL', 'Beaver Creek Snotel Site')

### Prediction

#### Predict discharge based on the snotel data

In [14]:
# the dataframe in case I need it, good for visualization 
snotel_df

ACCUMULATED PRECIPITATION  SWE  AIR TEMP  \
date                site                                                    
1987-10-22 06:00:00 817:WA:SNTL                        0.0  NaN    -60.34   
                    990:WA:SNTL                        NaN  NaN       NaN   
1987-10-22 08:00:00 817:WA:SNTL                        NaN  NaN    -60.34   
                    990:WA:SNTL                        NaN  NaN       NaN   
1987-10-22 11:00:00 817:WA:SNTL                        0.0  NaN    -60.34   
...                                                    ...  ...       ...   
2024-01-01 06:00:00 990:WA:SNTL                       24.1  6.4     30.02   
2024-01-01 07:00:00 817:WA:SNTL                       29.4  9.1     25.34   
                    990:WA:SNTL                       24.1  6.3     30.20   
2024-01-01 08:00:00 817:WA:SNTL                       29.4  9.1       NaN   
                    990:WA:SNTL                       24.1  6.3       NaN   

                                 SNOWDEPTH  
date                site                    
1987-10-22 06:00:00 817:WA:SNTL        NaN  
                    990:WA:SNTL        NaN  
1987-10-22 08:00:00 817:WA:SNTL        NaN  
                    990:WA:SNTL        NaN  
1987-10-22 11:00:00 817:WA:SNTL        NaN  
...                                    ...  
2024-01-01 06:00:00 990:WA:SNTL       22.0  
2024-01-01 07:00:00 817:WA:SNTL       21.0  
                    990:WA:SNTL       22.0  
2024-01-01 08:00:00 817:WA:SNTL       21.0  
                    990:WA:SNTL       22.0  

[437896 rows x 4 columns]

In [ ]:
# do some cleaning, we've got some null values 



In [ ]:
# fit a model based on air temp, SWE, and snow_depth to predict discharge 

### Classification

In [36]:
# set grounds truths

# Snow or Rain on Snow: binary classification
# 0: Snow 
# 1: Rain on Snow 

ground_truth = []



### We will start with a basic rolling mean classification....

In [ ]:
# collect 

def class_stats(X,y):
    

In [16]:
def assign_label(data):
    mu = data['DISCHARGE'].rolling(date=24*7, center=True).median()
    sigma = data['DISCHARGE'].rolling(date=24*7, center=True).std()

    
    

In [ ]:
# will use sklearn's decision tree for now....

from sklearn import tree
clf = tree.DecisionTreeClassifier()